In [1]:
using Distributions

function poisson_dgp(N, mean_traps)
    α = rand(Normal(log(4), .1))
    β = rand(Normal(-0.25, .1))
    traps = rand(Poisson(mean_traps), N)
    complaints = similar(traps)
    for i in eachindex(traps)
        complaints[i] = rand(Poisson(exp(α + β * traps[i])))
    end

    return (α = α, β = β, traps = traps, complaints = complaints)
end

poisson_dgp (generic function with 1 method)

In [2]:
simulated_poisson = poisson_dgp(100, 5);

In [16]:
using Turing

Turing.@model simple_poisson_turing(traps, complaints) = begin
    α ~ Normal(log(4), .1)
    β ~ Normal(-0.25, .1)

    for i in eachindex(traps)
        λ = exp(α + β * traps[i])
        complaints ~ Poisson(λ)
    end
end

DynamicPPL.ModelGen{var"###generator#332",(:traps, :complaints),(),Tuple{}}(##generator#332, NamedTuple())

In [15]:
args = (
    #μ_α = log(4), μ_β = -0.25,
    traps = simulated_poisson.traps, complaints = simulated_poisson.complaints)
#simple_poisson_turing(; args...)

turing_posterior = Turing.sample(simple_poisson_turing(; args...), NUTS(200, 0.65), 2500)

MethodError: MethodError: no method matching +(::Float64, ::Array{Float64,1})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(::Float64, !Matched::Float64) at float.jl:401
  +(::AbstractFloat, !Matched::Bool) at bool.jl:106
  ...